In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
dataset_path = '/content/drive/MyDrive/Leaf_Spot/Leaf Spot'
os.chdir(dataset_path)

In [ ]:
import os
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Load data
def load_data_from_directories(train_dir='train', test_dir='test'):
    train_image_paths = []
    train_labels = []

    # Process training data
    for cls in os.listdir(train_dir):
        cls_path = os.path.join(train_dir, cls)
        if os.path.isdir(cls_path):
            for img_name in os.listdir(cls_path):
                img_path = os.path.join(cls_path, img_name)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    train_image_paths.append(img_path)
                    class_number = int(cls.split()[-1]) - 1  # Convert to 0-indexed
                    train_labels.append(class_number)

    print(f"Found {len(train_image_paths)} training images")
    print(f"Class distribution: {Counter(train_labels)}")

    return train_image_paths, train_labels


def load_test_data_with_subdirs(test_dir='test'):
    test_image_paths = []
    test_labels = []
    label_mapping = {}
    current_label = 0

    # Process test data
    for cls in os.listdir(test_dir):
        cls_path = os.path.join(test_dir, cls)
        if os.path.isdir(cls_path):
            if cls not in label_mapping:
                label_mapping[cls] = current_label
                current_label += 1
            class_number = label_mapping[cls]

            for img_name in os.listdir(cls_path):
                img_path = os.path.join(cls_path, img_name)
                if img_path.lower().endswith(('.png', '.jpg', '.jpeg')):
                    test_image_paths.append(img_path)
                    test_labels.append(class_number)

    print(f"Found {len(test_image_paths)} test images")
    print(f"Class distribution: {Counter(test_labels)}")

    return test_image_paths, test_labels

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from tqdm import tqdm

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(16, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))  # Global average pooling

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        return x.view(x.size(0), -1)

# Load model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
simple_cnn = SimpleCNN().to(device)
simple_cnn.eval()

# Define transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
])

# Custom Dataset for feature extraction
class FeatureDataset(Dataset):
    def __init__(self, image_paths, transform=None):
        self.image_paths = image_paths
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image

# Extract features
def extract_features(model, image_paths, batch_size=32):
    dataset = FeatureDataset(image_paths, transform=transform)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    features = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Extracting features"):
            batch = batch.to(device)
            batch_features = model(batch)
            features.append(batch_features.cpu().numpy())

    return np.vstack(features)


# Load image paths
train_image_paths, train_labels = load_data_from_directories(train_dir='train')
test_image_paths, test_labels = load_test_data_with_subdirs(test_dir='test')

Found 242 training images
Class distribution: Counter({5: 57, 4: 51, 2: 43, 1: 36, 0: 32, 3: 23})
Found 31 test images
Class distribution: Counter({5: 8, 0: 5, 2: 5, 4: 5, 1: 4, 3: 4})


In [ ]:
# Extract Simple CNN features
train_simple_cnn = extract_features(simple_cnn, train_image_paths)
test_simple_cnn = extract_features(simple_cnn, test_image_paths)

Extracting features: 100%|██████████| 1/1 [00:27<00:00, 27.19s/it]


In [ ]:
import torchvision.models as models

# Load pre-trained models
def get_pretrained_model(name):
    if name == "densenet":
        model = models.densenet121(pretrained=True)
        model.classifier = nn.Identity()  # Remove classification head
    elif name == "mobilenet":
        model = models.mobilenet_v2(pretrained=True)
        model.classifier = nn.Identity()
    elif name == "resnet":
        model = models.resnet18(pretrained=True)
        model.fc = nn.Identity()
    model.eval()
    return model.to(device)

# Extract DenseNet features
densenet = get_pretrained_model("densenet")
train_densenet = extract_features(densenet, train_image_paths)
test_densenet = extract_features(densenet, test_image_paths)

# Extract MobileNet features
mobilenet = get_pretrained_model("mobilenet")
train_mobilenet = extract_features(mobilenet, train_image_paths)
test_mobilenet = extract_features(mobilenet, test_image_paths)

# Extract ResNet features
resnet = get_pretrained_model("resnet")
train_resnet = extract_features(resnet, train_image_paths)
test_resnet = extract_features(resnet, test_image_paths)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 103MB/s]
Extracting features: 100%|██████████| 1/1 [00:14<00:00, 14.43s/it]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a wei

In [ ]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

def train_and_evaluate(model, X_train, y_train, X_test, y_test):
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return accuracy_score(y_test, preds)

In [ ]:
# Example: Train SVM on ResNet features
svm = SVC(kernel='rbf', class_weight='balanced')
accuracy = train_and_evaluate(svm, train_resnet, train_labels, test_resnet, test_labels)
print(f"SVM Accuracy (ResNet): {accuracy:.2f}")

SVM Accuracy (ResNet): 0.58


In [ ]:
models = {
    # Traditional ML
    "SVM": SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42),
    "SVM-poly": SVC(kernel='poly', class_weight='balanced', probability=True, random_state=42),
    "SVM-linear": SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42),
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),

    # Ensemble Learning
    "Random Forest": RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    # "AdaBoost": AdaBoostClassifier(
    #     base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'),
    #     n_estimators=50,
    #     random_state=42
    # ),
    # "XGBoost": XGBClassifier(
    #     use_label_encoder=False,
    #     eval_metric='mlogloss',
    #     scale_pos_weight=1,  # Adjust if binary classification
    #     random_state=42
    # ),

    # Meta-Ensemble
    # "Voting Classifier": VotingClassifier(
    #     estimators=[
    #         ('svm', SVC(kernel='rbf', class_weight='balanced', probability=True)),
    #         ('rf', RandomForestClassifier(n_estimators=100, class_weight='balanced')),
    #         ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    #     ],
    #     voting='soft'
    # )
}

## CNN


In [ ]:
results = []

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(train_simple_cnn, train_labels)
    preds = model.predict(test_simple_cnn)

    accuracy = accuracy_score(test_labels, preds)
    report = classification_report(test_labels, preds, output_dict=True)


    results.append({
        "Model": name,
        "Accuracy": round(accuracy, 2),
        "Precision": round(report['weighted avg']['precision'], 2),
        "Recall": round(report['weighted avg']['recall'], 2),
        "F1-Score": round(report['weighted avg']['f1-score'], 2)
    })

    print(f"\n{name} Results:")
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(test_labels, preds))
    print("-" * 60)

Training SVM...

SVM Results:
Accuracy: 0.42
              precision    recall  f1-score   support

           0       0.67      0.40      0.50         5
           1       0.23      0.75      0.35         4
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         4
           4       1.00      0.20      0.33         5
           5       0.58      0.88      0.70         8

    accuracy                           0.42        31
   macro avg       0.41      0.37      0.31        31
weighted avg       0.45      0.42      0.36        31

------------------------------------------------------------
Training SVM-poly...


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


SVM-poly Results:
Accuracy: 0.42
              precision    recall  f1-score   support

           0       0.33      0.40      0.36         5
           1       0.14      0.25      0.18         4
           2       0.29      0.40      0.33         5
           3       0.50      0.50      0.50         4
           4       1.00      0.20      0.33         5
           5       0.83      0.62      0.71         8

    accuracy                           0.42        31
   macro avg       0.52      0.40      0.40        31
weighted avg       0.56      0.42      0.44        31

------------------------------------------------------------
Training SVM-linear...

SVM-linear Results:
Accuracy: 0.13
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         5
           1       0.13      1.00      0.23         4
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00         4
           4       0.00      0.0

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/m


K-Nearest Neighbors Results:
Accuracy: 0.65
              precision    recall  f1-score   support

           0       0.50      0.80      0.62         5
           1       0.33      0.25      0.29         4
           2       0.60      0.60      0.60         5
           3       0.80      1.00      0.89         4
           4       0.75      0.60      0.67         5
           5       0.83      0.62      0.71         8

    accuracy                           0.65        31
   macro avg       0.64      0.65      0.63        31
weighted avg       0.66      0.65      0.64        31

------------------------------------------------------------
Training Random Forest...

Random Forest Results:
Accuracy: 0.77
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       0.60      0.75      0.67         4
           2       1.00      0.60      0.75         5
           3       0.60      0.75      0.67         4
           4   

In [ ]:
import pandas as pd

# Convert results to DataFrame
df_results = pd.DataFrame(results)
print("Model Comparison:")
print(df_results)

Model Comparison:
                 Model  Accuracy  Precision  Recall  F1-Score
0                  SVM      0.42       0.45    0.42      0.36
1             SVM-poly      0.42       0.56    0.42      0.44
2           SVM-linear      0.13       0.02    0.13      0.03
3  Logistic Regression      0.29       0.24    0.29      0.25
4  K-Nearest Neighbors      0.65       0.66    0.65      0.64
5        Random Forest      0.77       0.79    0.77      0.77


## ResNet

In [ ]:
results = []

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(train_resnet, train_labels)
    preds = model.predict(test_resnet)

    accuracy = accuracy_score(test_labels, preds)
    report = classification_report(test_labels, preds, output_dict=True)


    results.append({
        "Model": name,
        "Accuracy": round(accuracy, 2),
        "Precision": round(report['weighted avg']['precision'], 2),
        "Recall": round(report['weighted avg']['recall'], 2),
        "F1-Score": round(report['weighted avg']['f1-score'], 2)
    })

    print(f"\n{name} Results:")
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(test_labels, preds))
    print("-" * 60)

Training SVM...

SVM Results:
Accuracy: 0.58
              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.43      0.75      0.55         4
           2       0.40      0.40      0.40         5
           3       0.60      0.75      0.67         4
           4       1.00      0.40      0.57         5
           5       0.75      0.75      0.75         8

    accuracy                           0.58        31
   macro avg       0.61      0.57      0.56        31
weighted avg       0.63      0.58      0.58        31

------------------------------------------------------------
Training SVM-poly...

SVM-poly Results:
Accuracy: 0.61
              precision    recall  f1-score   support

           0       0.50      0.40      0.44         5
           1       0.33      0.50      0.40         4
           2       0.67      0.40      0.50         5
           3       0.50      0.75      0.60         4
           4       1.00  

In [ ]:
import pandas as pd

# Convert results to DataFrame
df_results = pd.DataFrame(results)
print("Model Comparison:")
print(df_results)

Model Comparison:
                 Model  Accuracy  Precision  Recall  F1-Score
0                  SVM      0.58       0.63    0.58      0.58
1             SVM-poly      0.61       0.66    0.61      0.61
2           SVM-linear      0.71       0.71    0.71      0.70
3  Logistic Regression      0.68       0.70    0.68      0.68
4  K-Nearest Neighbors      0.48       0.60    0.48      0.47
5        Random Forest      0.55       0.58    0.55      0.53


## MobileNet

In [ ]:
results = []

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(train_mobilenet, train_labels)
    preds = model.predict(test_mobilenet)

    accuracy = accuracy_score(test_labels, preds)
    report = classification_report(test_labels, preds, output_dict=True)


    results.append({
        "Model": name,
        "Accuracy": round(accuracy, 2),
        "Precision": round(report['weighted avg']['precision'], 2),
        "Recall": round(report['weighted avg']['recall'], 2),
        "F1-Score": round(report['weighted avg']['f1-score'], 2)
    })

    print(f"\n{name} Results:")
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(test_labels, preds))
    print("-" * 60)

Training SVM...

SVM Results:
Accuracy: 0.61
              precision    recall  f1-score   support

           0       0.50      0.60      0.55         5
           1       0.33      0.75      0.46         4
           2       1.00      0.40      0.57         5
           3       0.75      0.75      0.75         4
           4       1.00      0.40      0.57         5
           5       0.75      0.75      0.75         8

    accuracy                           0.61        31
   macro avg       0.72      0.61      0.61        31
weighted avg       0.74      0.61      0.62        31

------------------------------------------------------------
Training SVM-poly...

SVM-poly Results:
Accuracy: 0.55
              precision    recall  f1-score   support

           0       0.60      0.60      0.60         5
           1       0.00      0.00      0.00         4
           2       1.00      0.40      0.57         5
           3       1.00      0.50      0.67         4
           4       1.00  

In [ ]:
import pandas as pd

# Convert results to DataFrame
df_results = pd.DataFrame(results)
print("Model Comparison:")
print(df_results)

Model Comparison:
                 Model  Accuracy  Precision  Recall  F1-Score
0                  SVM      0.61       0.74    0.61      0.62
1             SVM-poly      0.55       0.68    0.55      0.54
2           SVM-linear      0.71       0.75    0.71      0.71
3  Logistic Regression      0.61       0.64    0.61      0.62
4  K-Nearest Neighbors      0.58       0.69    0.58      0.59
5        Random Forest      0.58       0.69    0.58      0.57


## DenseNet

In [ ]:
results = []

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(train_densenet, train_labels)
    preds = model.predict(test_densenet)

    accuracy = accuracy_score(test_labels, preds)
    report = classification_report(test_labels, preds, output_dict=True)


    results.append({
        "Model": name,
        "Accuracy": round(accuracy, 2),
        "Precision": round(report['weighted avg']['precision'], 2),
        "Recall": round(report['weighted avg']['recall'], 2),
        "F1-Score": round(report['weighted avg']['f1-score'], 2)
    })

    print(f"\n{name} Results:")
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(test_labels, preds))
    print("-" * 60)

Training SVM...

SVM Results:
Accuracy: 0.58
              precision    recall  f1-score   support

           0       0.50      0.20      0.29         5
           1       0.30      0.75      0.43         4
           2       0.67      0.40      0.50         5
           3       0.67      0.50      0.57         4
           4       1.00      0.60      0.75         5
           5       0.70      0.88      0.78         8

    accuracy                           0.58        31
   macro avg       0.64      0.55      0.55        31
weighted avg       0.65      0.58      0.58        31

------------------------------------------------------------
Training SVM-poly...

SVM-poly Results:
Accuracy: 0.65
              precision    recall  f1-score   support

           0       1.00      0.20      0.33         5
           1       0.33      1.00      0.50         4
           2       1.00      0.40      0.57         5
           3       1.00      0.50      0.67         4
           4       1.00  

In [ ]:
import pandas as pd

# Convert results to DataFrame
df_results = pd.DataFrame(results)
print("Model Comparison on DenseNet features:")
print(df_results)

Model Comparison on DenseNet features:
                 Model  Accuracy  Precision  Recall  F1-Score
0                  SVM      0.58       0.65    0.58      0.58
1             SVM-poly      0.65       0.84    0.65      0.63
2           SVM-linear      0.77       0.81    0.77      0.78
3  Logistic Regression      0.77       0.81    0.77      0.78
4  K-Nearest Neighbors      0.45       0.62    0.45      0.49
5        Random Forest      0.61       0.72    0.61      0.61


# More models

In [ ]:
import torchvision.models as models

# Function to load pre-trained models
def get_pretrained_model(name):
    if name == "resnext":
        model = models.resnext50_32x4d(pretrained=True)
        model.fc = nn.Identity()  # Remove classification head
    elif name == "vgg16":
        model = models.vgg16(pretrained=True)
        model.classifier[-1] = nn.Identity()  # Remove classification head
    elif name == "vgg19":
        model = models.vgg19(pretrained=True)
        model.classifier[-1] = nn.Identity()
    model.eval()
    return model.to(device)

# Load models
resnext = get_pretrained_model("resnext")
vgg16 = get_pretrained_model("vgg16")
vgg19 = get_pretrained_model("vgg19")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNeXt50_32X4D_Weights.IMAGENET1K_V1`. You can also use `weights=ResNeXt50_32X4D_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnext50_32x4d-7cdf4587.pth" to /root/.cache/torch/hub/checkpoints/resnext50_32x4d-7cdf4587.pth
100%|██████████| 95.8M/95.8M [00:01<00:00, 90.1MB/s]
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated si

In [ ]:
# Extract ResNeXt features
train_resnext = extract_features(resnext, train_image_paths)
test_resnext = extract_features(resnext, test_image_paths)

# Extract VGG16 features
train_vgg16 = extract_features(vgg16, train_image_paths)
test_vgg16 = extract_features(vgg16, test_image_paths)

# Extract VGG19 features
train_vgg19 = extract_features(vgg19, train_image_paths)
test_vgg19 = extract_features(vgg19, test_image_paths)

print(f"ResNeXt feature shape: {train_resnext.shape}")  # (n_samples, 2048)
print(f"VGG16 feature shape: {train_vgg16.shape}")      # (n_samples, 4096)
print(f"VGG19 feature shape: {train_vgg19.shape}")      # (n_samples, 4096)

Extracting features: 100%|██████████| 1/1 [00:24<00:00, 24.45s/it]

ResNeXt feature shape: (242, 2048)
VGG16 feature shape: (242, 4096)
VGG19 feature shape: (242, 4096)


In [ ]:
models = {
    # Traditional ML
    "SVM": SVC(kernel='rbf', class_weight='balanced', probability=True, random_state=42),
    "SVM-poly": SVC(kernel='poly', class_weight='balanced', probability=True, random_state=42),
    "SVM-linear": SVC(kernel='linear', class_weight='balanced', probability=True, random_state=42),
    "Logistic Regression": LogisticRegression(class_weight='balanced', max_iter=1000, random_state=42),
    "K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=5),

    # Ensemble Learning
    "Random Forest": RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42),
    # "AdaBoost": AdaBoostClassifier(
    #     base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'),
    #     n_estimators=50,
    #     random_state=42
    # ),
    # "XGBoost": XGBClassifier(
    #     use_label_encoder=False,
    #     eval_metric='mlogloss',
    #     scale_pos_weight=1,  # Adjust if binary classification
    #     random_state=42
    # ),

    # Meta-Ensemble
    # "Voting Classifier": VotingClassifier(
    #     estimators=[
    #         ('svm', SVC(kernel='rbf', class_weight='balanced', probability=True)),
    #         ('rf', RandomForestClassifier(n_estimators=100, class_weight='balanced')),
    #         ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'))
    #     ],
    #     voting='soft'
    # )
}

## Resnext

In [ ]:
results = []

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(train_resnext, train_labels)
    preds = model.predict(test_resnext)

    accuracy = accuracy_score(test_labels, preds)
    report = classification_report(test_labels, preds, output_dict=True)


    results.append({
        "Model": name,
        "Accuracy": round(accuracy, 2),
        "Precision": round(report['weighted avg']['precision'], 2),
        "Recall": round(report['weighted avg']['recall'], 2),
        "F1-Score": round(report['weighted avg']['f1-score'], 2)
    })

    print(f"\n{name} Results:")
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(test_labels, preds))
    print("-" * 60)

Training SVM...

SVM Results:
Accuracy: 0.45
              precision    recall  f1-score   support

           0       0.25      0.20      0.22         5
           1       0.44      1.00      0.62         4
           2       0.25      0.20      0.22         5
           3       0.40      0.50      0.44         4
           4       1.00      0.40      0.57         5
           5       0.57      0.50      0.53         8

    accuracy                           0.45        31
   macro avg       0.49      0.47      0.43        31
weighted avg       0.50      0.45      0.44        31

------------------------------------------------------------
Training SVM-poly...

SVM-poly Results:
Accuracy: 0.61
              precision    recall  f1-score   support

           0       0.40      0.40      0.40         5
           1       0.60      0.75      0.67         4
           2       1.00      0.40      0.57         5
           3       1.00      0.50      0.67         4
           4       1.00  

In [ ]:
import pandas as pd

# Convert results to DataFrame
df_results = pd.DataFrame(results)
print("Model Comparison on Resnext features:")
print(df_results)

Model Comparison on Resnext features:
                 Model  Accuracy  Precision  Recall  F1-Score
0                  SVM      0.45       0.50    0.45      0.44
1             SVM-poly      0.61       0.73    0.61      0.60
2           SVM-linear      0.65       0.66    0.65      0.64
3  Logistic Regression      0.58       0.61    0.58      0.58
4  K-Nearest Neighbors      0.52       0.61    0.52      0.51
5        Random Forest      0.61       0.66    0.61      0.60


## VGG16

In [ ]:
results = []

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(train_vgg16, train_labels)
    preds = model.predict(test_vgg16)

    accuracy = accuracy_score(test_labels, preds)
    report = classification_report(test_labels, preds, output_dict=True)


    results.append({
        "Model": name,
        "Accuracy": round(accuracy, 2),
        "Precision": round(report['weighted avg']['precision'], 2),
        "Recall": round(report['weighted avg']['recall'], 2),
        "F1-Score": round(report['weighted avg']['f1-score'], 2)
    })

    print(f"\n{name} Results:")
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(test_labels, preds))
    print("-" * 60)

Training SVM...

SVM Results:
Accuracy: 0.65
              precision    recall  f1-score   support

           0       0.67      0.80      0.73         5
           1       0.38      0.75      0.50         4
           2       0.50      0.40      0.44         5
           3       1.00      0.75      0.86         4
           4       1.00      0.40      0.57         5
           5       0.75      0.75      0.75         8

    accuracy                           0.65        31
   macro avg       0.72      0.64      0.64        31
weighted avg       0.72      0.65      0.65        31

------------------------------------------------------------
Training SVM-poly...

SVM-poly Results:
Accuracy: 0.68
              precision    recall  f1-score   support

           0       0.75      0.60      0.67         5
           1       0.38      0.75      0.50         4
           2       0.60      0.60      0.60         5
           3       1.00      0.75      0.86         4
           4       1.00  

In [ ]:
import pandas as pd

# Convert results to DataFrame
df_results = pd.DataFrame(results)
print("Model Comparison on VGG16 features:")
print(df_results)

Model Comparison on VGG16 features:
                 Model  Accuracy  Precision  Recall  F1-Score
0                  SVM      0.65       0.72    0.65      0.65
1             SVM-poly      0.68       0.75    0.68      0.69
2           SVM-linear      0.68       0.75    0.68      0.66
3  Logistic Regression      0.74       0.78    0.74      0.74
4  K-Nearest Neighbors      0.45       0.55    0.45      0.45
5        Random Forest      0.65       0.73    0.65      0.65


## VGG19

In [ ]:
results = []

for name, model in models.items():
    print(f"Training {name}...")
    model.fit(train_vgg19, train_labels)
    preds = model.predict(test_vgg19)

    accuracy = accuracy_score(test_labels, preds)
    report = classification_report(test_labels, preds, output_dict=True)


    results.append({
        "Model": name,
        "Accuracy": round(accuracy, 2),
        "Precision": round(report['weighted avg']['precision'], 2),
        "Recall": round(report['weighted avg']['recall'], 2),
        "F1-Score": round(report['weighted avg']['f1-score'], 2)
    })

    print(f"\n{name} Results:")
    print(f"Accuracy: {accuracy:.2f}")
    print(classification_report(test_labels, preds))
    print("-" * 60)

Training SVM...

SVM Results:
Accuracy: 0.65
              precision    recall  f1-score   support

           0       0.60      0.60      0.60         5
           1       0.44      1.00      0.62         4
           2       0.50      0.40      0.44         5
           3       0.80      1.00      0.89         4
           4       1.00      0.40      0.57         5
           5       0.83      0.62      0.71         8

    accuracy                           0.65        31
   macro avg       0.70      0.67      0.64        31
weighted avg       0.71      0.65      0.64        31

------------------------------------------------------------
Training SVM-poly...

SVM-poly Results:
Accuracy: 0.74
              precision    recall  f1-score   support

           0       1.00      0.80      0.89         5
           1       0.44      1.00      0.62         4
           2       0.67      0.40      0.50         5
           3       0.80      1.00      0.89         4
           4       1.00  

In [ ]:
import pandas as pd

# Convert results to DataFrame
df_results = pd.DataFrame(results)
print("Model Comparison on VGG19 features:")
print(df_results)

Model Comparison on VGG19 features:
                 Model  Accuracy  Precision  Recall  F1-Score
0                  SVM      0.65       0.71    0.65      0.64
1             SVM-poly      0.74       0.81    0.74      0.75
2           SVM-linear      0.71       0.72    0.71      0.71
3  Logistic Regression      0.71       0.70    0.71      0.69
4  K-Nearest Neighbors      0.58       0.64    0.58      0.56
5        Random Forest      0.58       0.68    0.58      0.60
